In [15]:
import torch
import torch.nn as nn
import torch.optim as optim


In [16]:

# Define the training examples
training_examples = [
    "The quick brown fox jumps over the lazy dog",
    "She sells seashells by the seashore",
    "Peter Piper picked a peck of pickled peppers",
    "How much wood would a woodchuck chuck if a woodchuck could chuck wood",
    "I scream, you scream, we all scream for ice cream"
]

In [17]:
# Define the vocabulary
vocab = set("".join(training_examples))
vocab_size = len(vocab)
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for idx, char in enumerate(vocab)}


In [18]:
# Define the model
class TextCompletionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TextCompletionModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

In [19]:

# Define the hyperparameters
input_size = vocab_size
hidden_size = 64
output_size = vocab_size
learning_rate = 0.01
num_epochs = 100

In [20]:
# Initialize the model and optimizer
model = TextCompletionModel(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:

# Train the model
for epoch in range(num_epochs):
    for example in training_examples:
        # Convert the example to a tensor
        input_tensor = torch.zeros(len(example), input_size)
        for i, char in enumerate(example):
            input_tensor[i, char_to_idx[char]] = 1
            
        # Initialize the hidden state
        hidden = model.init_hidden(1)
        
        # Forward pass
        output, hidden = model(input_tensor.unsqueeze(0), hidden)
        
        # Compute the loss
        loss = nn.CrossEntropyLoss()(output.squeeze(0), input_tensor.argmax(dim=1))
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Print the loss every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")



Epoch 0, Loss: 2.960742950439453
Epoch 10, Loss: 0.12615516781806946
Epoch 20, Loss: 0.016170619055628777
Epoch 30, Loss: 0.0074917045421898365
Epoch 40, Loss: 0.004737526644021273
Epoch 50, Loss: 0.003328861901536584
Epoch 60, Loss: 0.002490279031917453
Epoch 70, Loss: 0.0019439209718257189
Epoch 80, Loss: 0.0015652687288820744
Epoch 90, Loss: 0.001290685497224331


In [26]:
# Test the model
test_example = "The quick brown"
input_tensor = torch.zeros(len(test_example), input_size)
for i, char in enumerate(test_example):
    input_tensor[i, char_to_idx[char]] = 1
hidden = model.init_hidden(1)
output, hidden = model(input_tensor.unsqueeze(0), hidden)
predicted_chars = []
for idx in output.argmax(dim=2).squeeze():
    if idx.item() < vocab_size:
        predicted_chars.append(idx_to_char[idx.item()])
    else:
        predicted_chars.append("<UNK>")
predicted_text = test_example + "".join(predicted_chars[len(test_example):])
print(f"Predicted text: {predicted_text}")


Predicted text: The quick brown


Train more with diverse examples to increase accuracy